In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, item_knn as knn
from lenskit import topn
import pandas as pd
%matplotlib inline

In [ ]:
ratings = pd.read_csv('dataSets/jester_dataset_2/jester_ratings.dat', sep='\t\t' , names=['user', 'item', 'rating'])
ratings.head()

In [3]:
ratings.rating += 10
print(ratings)

          user  item  rating
0            1     5  10.219
1            1     7   0.719
2            1     8   0.719
3            1    13   3.219
4            1    15  10.875
5            1    16   0.344
6            1    17   0.969
7            1    18   2.531
8            1    19   1.281
9            1    20   0.844
10           1    21   2.812
11           1    22   1.219
12           1    23   1.469
13           1    24   2.094
14           1    25   2.531
15           1    89  19.812
16           1    50  19.906
17           1   102  10.750
18           1   103   5.000
19           1   104  12.938
20           1   105  12.000
21           1   106   9.844
22           1   107  12.031
23           1   108  15.688
24           1   109  19.656
25           1    87  18.000
26           1    93  19.312
27           1    76  19.312
28           1    65  18.781
29           1    72  18.781
...        ...   ...     ...
1761409  63978    81  17.531
1761410  63978   142  17.844
1761411  63978

In [4]:
algo_ii = knn.ItemItem(20)
algo_als = als.BiasedMF(50)

In [7]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    # now we run the recommender
    recs = batch.recommend(fittable, users, 10)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [ ]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))

In [ ]:
all_recs = pd.concat(all_recs, ignore_index=True)
print(all_recs)

In [ ]:
test_data = pd.concat(test_data, ignore_index=True)

In [ ]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg)
results = rla.compute(all_recs, test_data)
print(results)

In [ ]:
results.groupby('Algorithm').ndcg.mean()